# Battle of the Neighborhoods

importing a libaraies 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab
usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


# Download and Explore Toronto Dataset

The dateset being used is found at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

The dataset is a list of Toronto's zipcodes which includes the boroughs and neighborhood names.

In [2]:

#Obtain Postal Code, Borough, and Neighborhood information from Wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#Obtain the first table
df_toronto = table[0]
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Transform the data

In [3]:
df_toronto.rename(columns = {"Postal Code": "PostalCode","Neighbourhood": "Neighborhood"}, inplace = True)

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)
#df.head()

#Combine the neighborhoods that exists in one postal code
df_toronto = df_toronto.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
df_toronto .head()

#Change unassigned Neighborhood to its Borough's name
df_toronto.loc[85,'Neighborhood'] = 'Queen\'s Park'

print (df_toronto.shape)

df_toronto.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Join neighborhood table with latitude and longitude information

In [4]:
#Create a dataframe of the latitude and longitudes of the Toronto Neighborhoods
latlong = pd.read_csv("http://cocl.us/Geospatial_data")
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
latlong.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
latlong.shape

(103, 3)

In [7]:
#Join the Lat and Long dataframe to Neighborhoods dataframe
df_toronto.set_index("PostalCode")
latlong.set_index("PostalCode")
neighbor=pd.merge(df_toronto, latlong)
neighbor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [8]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(len(neighbor['Borough'].unique()),neighbor.shape[0]))

Toronto has 10 boroughs and 103 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto, Canada

In [9]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top

In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
scarborough_data = neighbor[neighbor['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
scarborough_data.shape

(17, 5)

Lets get the geographical coordinates of Scarborough

In [13]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough, CA are 43.773077, -79.257774.


In [14]:
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

# Explore Scarborough neighborhood in Toronto with Foursquare API

### Define Foursquare credentials and version

In [15]:
CLIENT_ID = 'xxxxxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## Function to explore neighborhoods

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [17]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],latitudes=scarborough_data['Latitude'],longitudes=scarborough_data['Longitude'])

Found 94 venues in 17 neighborhoods.


In [18]:
print(scarborough_venues.shape)
scarborough_venues.head()

(94, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [19]:
#Venues per Neighborhood
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",8,8,8,8,8,8
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [20]:
print('There are {} distinct venues in {} categories.'.format(
    len(scarborough_venues['Venue'].unique()),len(scarborough_venues['Venue Category'].unique())))

#print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 85 distinct venues in 57 categories.


In [21]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Health & Beauty Service,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Mexican Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Health & Beauty Service,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Mexican Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.2,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.2,0.000,0.200000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.000,0.000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000,0.000
1,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.000000,0.25,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.000,0.000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000,0.000
2,Cedarbrae,0.000000,0.111111,0.000000,0.111111,0.111111,0.0,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.111111,0.000000,0.0,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.111111,0.111111,0.00,0.000000,0.111111,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000,0.111111,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000,0.000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000000,0.000000,0.000000,0.083333,0.0,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.083333,0.0,0.000000,0.00,0.00,0.00,0.000000,0.083333,0.083333,0.083333,0.00,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.083333,0.083333,0.00,0.0,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.083333,0.000000,0.000,0.083333,0.166667,0.0,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000,0.000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000,0.000,0.333333,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.000,0.000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000,0.000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.125,0.000000,0.000000,0.00,0.000000,0.000000,0.125000,0.0,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.25,0.000000,0.000000,0.00,0.0,0.125,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.0,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.125
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000000,0.000000,0.222222,0.000000,0.0,0.000000,0.000,0.222222,0.111111,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.0,0.111111,0.00,0.111111,0.000000,0.00,0.0,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.111111,0.000,0.000000,0.000000,0.0,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000,

# Each neighborhood with the top 5 venues

In [23]:
num_top_venues = 10

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1  Latin American Restaurant   0.2
2             Breakfast Spot   0.2
3               Skating Rink   0.2
4                     Lounge   0.2
5                  Pet Store   0.0
6         Italian Restaurant   0.0
7          Korean Restaurant   0.0
8         Light Rail Station   0.0
9             Medical Center   0.0


----Birch Cliff, Cliffside West----
                       venue  freq
0            College Stadium  0.25
1               Skating Rink  0.25
2      General Entertainment  0.25
3                       Café  0.25
4        American Restaurant  0.00
5         Italian Restaurant  0.00
6          Korean Restaurant  0.00
7  Latin American Restaurant  0.00
8         Light Rail Station  0.00
9                     Lounge  0.00


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.11
1                Bakery  0.11
2                  Bank  0.11
3       Thai Restaurant  0.11
4   

Let's put that into a pandas dataframe

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new dataframe and display the top ten venues for each neighborhood

In [25]:

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Hakka Restaurant,Grocery Store,Gas Station,Fried Chicken Joint
2,Cedarbrae,Caribbean Restaurant,Bank,Gas Station,Fried Chicken Joint,Lounge,Hakka Restaurant,Bakery,Thai Restaurant,Athletics & Sports,College Stadium
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Noodle House,Bank,Pharmacy,Italian Restaurant,Chinese Restaurant,Intersection,Fast Food Restaurant,Fried Chicken Joint,Gas Station
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Brewery,Thrift / Vintage Store,Light Rail Station,Pet Store,Chinese Restaurant,Breakfast Spot,Department Store,Athletics & Sports
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Bus Station,Soccer Field,Ice Cream Shop,Park,Intersection,Vietnamese Restaurant,Department Store,Discount Store
7,"Guildwood, Morningside, West Hill",Rental Car Location,Restaurant,Electronics Store,Intersection,Breakfast Spot,Mexican Restaurant,Medical Center,Bank,Fast Food Restaurant,Fried Chicken Joint
8,"Kennedy Park, Ionview, East Birchmount Park",Hobby Shop,Discount Store,Department Store,Coffee Shop,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
9,"Malvern, Rouge",Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store


In [26]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Rouge Hill, Port Union, Highland Creek
1st Most Common Venue                               Home Service
2nd Most Common Venue                                        Bar
3rd Most Common Venue                             Clothing Store
4th Most Common Venue                           Hakka Restaurant
5th Most Common Venue                              Grocery Store
6th Most Common Venue                      General Entertainment
7th Most Common Venue                                Gas Station
8th Most Common Venue                        Fried Chicken Joint
9th Most Common Venue                       Fast Food Restaurant
10th Most Common Venue                         Electronics Store
Name: 11, dtype: object

# Cluster the Scarborough Neighborhood using k-means

In [27]:
kclusters = 3

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0])

Lets create a new dataframe that includes the cluster as well as the top ten venues for each neighborhood

In [28]:
#Note that the neighborhood Upper Rouge does not have any venues, so I will drop from dataset
scarborough_data.drop(scarborough_data[scarborough_data.Neighborhood == 'Upper Rouge'].index, inplace = True)
#df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)

scarborough_merged = scarborough_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Home Service,Bar,Clothing Store,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Rental Car Location,Restaurant,Electronics Store,Intersection,Breakfast Spot,Mexican Restaurant,Medical Center,Bank,Fast Food Restaurant,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Caribbean Restaurant,Bank,Gas Station,Fried Chicken Joint,Lounge,Hakka Restaurant,Bakery,Thai Restaurant,Athletics & Sports,College Stadium


In [29]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Scarborough Neighborhood Cluster

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

#### Scarborough Clusters 0, 1, 2

In [30]:
scarborough_cluster_0 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_1 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_2 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

In [31]:
scarborough_cluster_0

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store
1,Scarborough,-79.160497,0,Home Service,Bar,Clothing Store,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Scarborough,-79.188711,0,Rental Car Location,Restaurant,Electronics Store,Intersection,Breakfast Spot,Mexican Restaurant,Medical Center,Bank,Fast Food Restaurant,Fried Chicken Joint
3,Scarborough,-79.216917,0,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,Scarborough,-79.239476,0,Caribbean Restaurant,Bank,Gas Station,Fried Chicken Joint,Lounge,Hakka Restaurant,Bakery,Thai Restaurant,Athletics & Sports,College Stadium
5,Scarborough,-79.239476,0,Health & Beauty Service,Playground,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,Scarborough,-79.262029,0,Hobby Shop,Discount Store,Department Store,Coffee Shop,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,Scarborough,-79.284577,0,Bakery,Bus Line,Bus Station,Soccer Field,Ice Cream Shop,Park,Intersection,Vietnamese Restaurant,Department Store,Discount Store
8,Scarborough,-79.239476,0,American Restaurant,Skating Rink,Motel,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,Scarborough,-79.264848,0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Hakka Restaurant,Grocery Store,Gas Station,Fried Chicken Joint


In [32]:
scarborough_cluster_1

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,-79.273304,1,Indian Restaurant,Vietnamese Restaurant,Brewery,Thrift / Vintage Store,Light Rail Station,Pet Store,Chinese Restaurant,Breakfast Spot,Department Store,Athletics & Sports
12,Scarborough,-79.262029,1,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


In [33]:
scarborough_cluster_2

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,-79.295849,2,Auto Garage,Smoke Shop,Bakery,Sandwich Place,Breakfast Spot,Fried Chicken Joint,Vietnamese Restaurant,Electronics Store,College Stadium,Department Store


# Explore New York City Neighborhoods

New York City has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. The link to the dataset: https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json

For your convenience, I downloaded the files and placed it on the server, so you can simply run a wget command and access the data. So let's go ahead and do that.

In [34]:
with open('nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

In [35]:
neighborhoods_data = newyork_data['features']

In [36]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

# Transform the data into a pandas dataframe

In [37]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [38]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [39]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [40]:
neighborhoods.shape

(306, 4)

In [41]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()),neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


# Use Geolibrary to get the latitude and longitude of New York City

In [42]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent ='ny-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [43]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Lets simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Queens neighborhood data.

In [44]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


Lets get the geographical location of Queens, N

In [45]:

geolocator = Nominatim(user_agent = 'ny-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7127281, -74.0060152.


In [46]:
# create map of Manhattan using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

# Lets explore the Long Island City neighborhood in Queens, NY

In [47]:
#Long Island City Latitude and Longitude values

neighborhood_latitude = queens_data.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = queens_data.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = queens_data.loc[10, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Long Island City are 40.75021734610528, -73.93920223915505.


# Top 100 venues in Long Island City neighborhood within a radius of 500 meters

First, let's create the GET request URL named url.

In [48]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [49]:
results = requests.get(url).json()

In [50]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [51]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\akmal\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Chip,Dessert Shop,40.750069,-73.940831
1,Etto Espresso Bar,Coffee Shop,40.748703,-73.940689
2,Baker House Market,Convenience Store,40.752137,-73.939235
3,Hilton Garden Inn,Hotel,40.750216,-73.936886
4,Dutch Kills,Cocktail Bar,40.747830,-73.940108


In [52]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

69 venues were returned by Foursquare.


# Analyze Each Neighborhood in Queens

In [53]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [54]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude'])

Found 2089 venues in 81 neighborhoods.


In [55]:
queens_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
3,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym
4,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop


In [56]:
print(queens_venues.shape)
queens_venues.tail()

(2089, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2084,Queensbridge,40.756091,-73.945631,Roosevelt Island Running Path,40.754902,-73.948958,Gym / Fitness Center
2085,Queensbridge,40.756091,-73.945631,Queensbridge Basketball Courts,40.755060,-73.949103,Basketball Court
2086,Queensbridge,40.756091,-73.945631,The Ravel Hotel Gym,40.753787,-73.948815,Athletics & Sports
2087,Queensbridge,40.756091,-73.945631,Estate Garden And Grill,40.753700,-73.948841,Beer Garden
2088,Queensbridge,40.756091,-73.945631,Track 114,40.753008,-73.947833,Platform


In [57]:
#Venues per Neighborhood
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,21,21,21,21,21,21
Astoria,100,100,100,100,100,100
Astoria Heights,12,12,12,12,12,12
Auburndale,22,22,22,22,22,22
Bay Terrace,39,39,39,39,39,39
Bayside,72,72,72,72,72,72
Bayswater,2,2,2,2,2,2
Beechhurst,14,14,14,14,14,14
Bellaire,10,10,10,10,10,10


In [58]:
print('There are {} distinct venues in {} categories.'.format(
    len(queens_venues['Venue'].unique()),len(queens_venues['Venue Category'].unique())))

There are 1715 distinct venues in 274 categories.


In [59]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
#scarborough_onehot = scarborough_onehot[fixed_columns]

neighbor = queens_onehot['Neighborhood']
queens_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
queens_onehot.insert(0, 'Neighborhood', neighbor)

queens_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga 

# Group by Neighborhood, and exame the frequency of the occurrence of venue

In [60]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga 

#### Each Neighborhood with the top 5 venues

In [61]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
            venue  freq
0       Surf Spot  0.19
1  Sandwich Place  0.10
2   Metro Station  0.10
3      Restaurant  0.05
4        Bus Stop  0.05


----Astoria----
                       venue  freq
0                        Bar  0.06
1  Middle Eastern Restaurant  0.06
2           Greek Restaurant  0.05
3   Mediterranean Restaurant  0.04
4                 Hookah Bar  0.04


----Astoria Heights----
                venue  freq
0  Italian Restaurant  0.08
1               Plaza  0.08
2        Burger Joint  0.08
3        Gourmet Shop  0.08
4              Bakery  0.08


----Auburndale----
                  venue  freq
0    Italian Restaurant  0.09
1        Discount Store  0.05
2            Hookah Bar  0.05
3        Gymnastics Gym  0.05
4  Fast Food Restaurant  0.05


----Bay Terrace----
            venue  freq
0  Clothing Store  0.10
1      Donut Shop  0.05
2  Cosmetics Shop  0.05
3            Bank  0.05
4  Lingerie Store  0.05


----Bayside----
               venue  freq
0     

                venue  freq
0        Dance Studio  0.06
1              Bakery  0.06
2  Italian Restaurant  0.06
3      Ice Cream Shop  0.06
4                Park  0.06


----Murray Hill----
               venue  freq
0  Korean Restaurant  0.47
1        Supermarket  0.04
2        Coffee Shop  0.04
3                Bar  0.04
4               Bank  0.04


----Neponsit----
               venue  freq
0              Beach   0.8
1                Bar   0.2
2  Accessories Store   0.0
3       Optical Shop   0.0
4               Park   0.0


----North Corona----
                  venue  freq
0         Deli / Bodega  0.19
1           Pizza Place  0.10
2  Gym / Fitness Center  0.10
3                Bakery  0.10
4           Coffee Shop  0.05


----Oakland Gardens----
                venue  freq
0   Korean Restaurant  0.14
1  Chinese Restaurant  0.11
2         Yoga Studio  0.07
3          Donut Shop  0.07
4                Bank  0.04


----Ozone Park----
         venue  freq
0     Pharmacy  0.09
1      

In [62]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Top venues for each neighborhood

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Metro Station,Sandwich Place,Bus Stop,Restaurant,Coffee Shop,Board Shop,Beach,Thai Restaurant,Donut Shop
1,Astoria,Bar,Middle Eastern Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Bakery,Food Truck,Seafood Restaurant,Pizza Place,Indian Restaurant
2,Astoria Heights,Bowling Alley,Plaza,Playground,Hostel,Burger Joint,Italian Restaurant,Pizza Place,Gourmet Shop,Bus Station,Business Service
3,Auburndale,Italian Restaurant,Hookah Bar,Athletics & Sports,Miscellaneous Shop,Fast Food Restaurant,Mattress Store,Furniture / Home Store,Supermarket,Noodle House,Sushi Restaurant
4,Bay Terrace,Clothing Store,Mobile Phone Shop,Women's Store,American Restaurant,Kids Store,Bank,Shoe Store,Lingerie Store,Cosmetics Shop,Donut Shop
5,Bayside,Bar,Indian Restaurant,Pub,American Restaurant,Pizza Place,Sushi Restaurant,Cosmetics Shop,Donut Shop,Ice Cream Shop,Pharmacy
6,Bayswater,Construction & Landscaping,Playground,Yoga Studio,Filipino Restaurant,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
7,Beechhurst,Yoga Studio,Supermarket,Italian Restaurant,Optical Shop,Park,Donut Shop,Pizza Place,Dessert Shop,Deli / Bodega,Chinese Restaurant
8,Bellaire,Breakfast Spot,Gym,Convenience Store,Coffee Shop,Chinese Restaurant,Bus Station,Moving Target,Greek Restaurant,Deli / Bodega,Italian Restaurant
9,Belle Harbor,Beach,Pub,Spa,Deli / Bodega,Bakery,Italian Restaurant,Mexican Restaurant,Pharmacy,Boutique,Chinese Restaurant


In [64]:
neighborhoods_venues_sorted.iloc[47,]

Neighborhood                Long Island City
1st Most Common Venue                  Hotel
2nd Most Common Venue            Coffee Shop
3rd Most Common Venue                    Bar
4th Most Common Venue            Pizza Place
5th Most Common Venue     Mexican Restaurant
6th Most Common Venue                   Café
7th Most Common Venue             Donut Shop
8th Most Common Venue            Supermarket
9th Most Common Venue          Deli / Bodega
10th Most Common Venue              Boutique
Name: 47, dtype: object

# Cluster the Queens Borough using K-Means

In [65]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Dataframe that includes the cluster of each neighborhood

In [66]:
queens_merged = queens_data
# add clustering labels
queens_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,0,Bar,Middle Eastern Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Bakery,Food Truck,Seafood Restaurant,Pizza Place,Indian Restaurant
1,Queens,Woodside,40.746349,-73.901842,0,Grocery Store,Bakery,Thai Restaurant,Filipino Restaurant,Latin American Restaurant,American Restaurant,Bar,Pub,Donut Shop,Gourmet Shop
2,Queens,Jackson Heights,40.751981,-73.882821,0,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Mexican Restaurant,Grocery Store,Thai Restaurant,Pizza Place,Supplement Shop
3,Queens,Elmhurst,40.744049,-73.881656,0,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant,Indonesian Restaurant,Pizza Place,Malay Restaurant,Bar,Bakery,Seafood Restaurant
4,Queens,Howard Beach,40.654225,-73.838138,0,Italian Restaurant,Bagel Shop,Pharmacy,Fast Food Restaurant,Sandwich Place,Clothing Store,Bar,Donut Shop,Tapas Restaurant,Sushi Restaurant


In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Queens Cluster Neighborhood

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

In [68]:
queens_cluster_0 = queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_1 = queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_2 = queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_3 = queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_4 = queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

In [69]:
queens_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,0,Bar,Middle Eastern Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Bakery,Food Truck,Seafood Restaurant,Pizza Place,Indian Restaurant
1,Woodside,0,Grocery Store,Bakery,Thai Restaurant,Filipino Restaurant,Latin American Restaurant,American Restaurant,Bar,Pub,Donut Shop,Gourmet Shop
2,Jackson Heights,0,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Mexican Restaurant,Grocery Store,Thai Restaurant,Pizza Place,Supplement Shop
3,Elmhurst,0,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant,Indonesian Restaurant,Pizza Place,Malay Restaurant,Bar,Bakery,Seafood Restaurant
4,Howard Beach,0,Italian Restaurant,Bagel Shop,Pharmacy,Fast Food Restaurant,Sandwich Place,Clothing Store,Bar,Donut Shop,Tapas Restaurant,Sushi Restaurant
5,Corona,0,Mexican Restaurant,Supermarket,Deli / Bodega,Empanada Restaurant,Chinese Restaurant,Sandwich Place,Basketball Court,South American Restaurant,Latin American Restaurant,Bakery
6,Forest Hills,0,Gym,Gym / Fitness Center,Yoga Studio,Asian Restaurant,Pharmacy,Convenience Store,Thai Restaurant,Park,Pizza Place,Liquor Store
7,Kew Gardens,0,Chinese Restaurant,Bar,Pet Store,Bank,Donut Shop,Pizza Place,Park,Cosmetics Shop,Indian Restaurant,Deli / Bodega
8,Richmond Hill,0,Bank,Pizza Place,Lounge,Latin American Restaurant,Spanish Restaurant,Diner,Pet Service,Discount Store,Park,Donut Shop
9,Flushing,0,Hotpot Restaurant,Bubble Tea Shop,Chinese Restaurant,Korean Restaurant,Bakery,Café,Gym,Seafood Restaurant,Construction & Landscaping,Szechuan Restaurant


In [70]:
queens_cluster_1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Utopia,1,Deli / Bodega,Automotive Shop,Playground,Pizza Place,Donut Shop,South American Restaurant,Spa,Locksmith,Grocery Store,Basketball Court


In [71]:
queens_cluster_2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,East Elmhurst,2,Donut Shop,Ice Cream Shop,Bus Station,Gym,Gas Station,Coffee Shop,Lake,Supermarket,Rental Car Location,Deli / Bodega
52,Floral Park,2,Indian Restaurant,Grocery Store,Basketball Court,Dosa Place,Pizza Place,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Flower Shop


In [72]:
queens_cluster_3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Cambria Heights,3,Caribbean Restaurant,Pharmacy,Restaurant,Bakery,Gym / Fitness Center,Nightclub,Liquor Store,Flower Shop,Fast Food Restaurant,Filipino Restaurant


In [73]:
queens_cluster_4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Glendale,4,Bus Station,Deli / Bodega,Brewery,Food & Drink Shop,Pizza Place,Arts & Crafts Store,Fish & Chips Shop,Farm,Farmers Market,Fast Food Restaurant
